[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jabascal/notes-on-machine-learning/blob/master/exploratory_analysis_embedding_tb.ipynb)

# Exploratory image analysis - Part 2 : Embeddings on tensorboard

Exploratory data analysis and visualization techniques are essential to get insight from the data. Unlock the full power of AI approaches by understanding and focusing on data quality!

For both Tensorflow and Pytorch users! 

Read article in Medium [here]()

In this notebook, we look at projection embeddings, which are useful techniques to uncover the underlying structure of high-dimensional data, such as images, by visualizing the data in 2D or 3D. We will use the `TensorBoard` library, as it allows for interactive visualization and provides out-of-the-box projections.

For an introduction to exploratory image analysis and for advanced density plots, read [Exploratory image analysis - Part 1](https://medium.com/@juanabascal78/exploratory-image-analysis-part-1-advanced-density-plots-19b255075dbd). 

## Introduction

What are projection embeddings and why are they useful? In machine learning, an embedding is a mapping of a discrete variable to a vector of continuous numbers. This vector space is a convenient data representation where we can perform operations such as distance calculations, clustering, and visualization. The projection implies that we are mapping high-dimensional data, such as images, into a low-dimensional space. What do we expect from the projection embedding? We expect that some of both local structure of the data (similar images are close to each other) and global structure (clusters of similar images) are preserved. By projecting the data into a 2D or 3D space, we can visualize the data and uncover the underlying structure. Thus, we say that these 2D or 3D representation of the data are embedded in the original high-dimensional space. 

How would you visualize images in 2D or 3D? One possibility would be to take two or three random pixels in the image and do a scatter plot, but which pixels to choose? A better choice would be to take two or three combination of pixels. There you have your embedding! 

In practice, the projection embedding in computed in two steps. First, we extract the features from the images using a pre-trained model, such as VGG16, ResNet, or MobileNet. Then, we use a dimensionality reduction technique to further reduce the number of dimensions to 2 or 3. We can differentiate between linear and non-linear dimensionality reduction techniques:  

- **Linear dimensionality reduction techniques**, such as principal components analysis (**PCA**) or non-negative matrix factorization (NMF) are widely used for this purpose. For instance, by choosing the first two principal components of the data, we can visualize the data in 2D. These two components would explain the maximum variability of the data, being orthogonal to each other. PCA works well for data that is linearly separable. 
- **Non-linear dimensionality reduction techniques** are used when the data is not linearly separable. Unlike linear methods they favor preserving the local distances between points. Standard methods are locally-linear embeddings (LLE), **UMAP** and **t-SNE**, which we will explore below. 

## Installation

First, we'll install the necessary libraries within an environment. 

In [ ]:
mode_colab = False
if mode_colab:
    !pip install tensorboard
else:
    !pip install matplotlib \
        numpy \
        tensorflow \
        tensorflow_hub

Tensorboard can be used within Tensorflow and Pytorch. For Pytorch, you don't need to install tensorflow, only `tensorboard`, as explained in [Run TensorBoard](https://pytorch.org/tutorials/recipes/recipes/tensorboard_with_pytorch.html).

In [ ]:
import os
import numpy as np
import PIL.Image as Image
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
from tensorboard.plugins import projector

np.random.seed(42)

## Data

We look at [CIFAR-10 dataset](https://www.cs.toronto.edu/~kriz/cifar.html), which is a collection 60,000 images of 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images. We will use the testset. Images have 32x32 pixels and 3 channels (RGB).

### Data download and loading

Data can be download from the [CIFAR-10 website](https://www.cs.toronto.edu/~kriz/cifar.html) or [Kaggle](https://www.kaggle.com/c/cifar-10/data), but the simplest is to download the data using keras or pytorch. We show how to download the data using keras, which requires to install tensorflow.

In [ ]:
from keras.datasets import cifar10

# Download the CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

# We retain only the test set
images = x_test
labels_ids = y_test

We define CIFAR-10 labels, as given on the website. 

In [ ]:
# Channels and CIFAR-10 classes
channels = ['r', 'g', 'b']
cifar10_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'] 
cifar10_labels_ids = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Labels names
labels_names = [cifar10_labels[label_id[0]] for label_id in labels_ids]

# Name for dataset (for saving results)
result_name = 'cifar10'

# Path for results
result_dir = '../Results/cifar10/data_anal/'    
if not os.path.exists(result_dir):
    os.makedirs(result_dir)

# Path for logging tensorboard embeddings
log_dir = os.path.join(result_dir, 'log_imgs')
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    print(f'Log directory created at {log_dir}')

After reading the data, we display few random images and then a sprite of the dataset. 

In [ ]:
def display_grid_images_labels(images, labels, dim_resize=None, num_subplots=(3, 3), 
                      figsize = (6, 6), path_file=None):
    """
    Load and display a grid of images from the specified list of image files.    
    """
    # Create a grid to display the images
    fig, axes = plt.subplots(num_subplots[0], num_subplots[1], figsize=figsize)
    axes = axes.flatten()  # Flatten the axes array
    # Load and display the selected images on the grid
    for i, (img, label) in enumerate(zip(images, labels)):        
        #ax = axes[i // num_subplots[0], i % num_subplots[1]]
        ax = axes[i]
        ax.imshow(img)
        ax.set_title(label)
        ax.axis('off')
    if path_file is not None:
        fig.savefig(path_file)
    else:
        plt.show()

num_selected = 16
images_selected = [images[i] for i in range(num_selected)]
labels_selected = [cifar10_labels[labels_ids[i][0]] for i in range(num_selected)]
file_save = os.path.join(result_dir, f'{result_name}_img.png')
display_grid_images_labels(images_selected, labels_selected, 
                           path_file=file_save, figsize=(5,6), num_subplots=(4,4))

To easily inspect an image:

In [ ]:
ind = 1000
print(f'Label: {cifar10_labels[labels_ids[ind][0]]}')
Image.fromarray(images[ind]).show()

We create a sprite plot of the dataset, which is a single image that contains all or a subset of the images in the dataset. We will use the sprite to visualize the embedding in an interactive fashion. 

In [ ]:
def images_to_sprite(data, invert_colors=False):
    if len(data.shape) == 3:
        data = np.tile(data[...,np.newaxis], (1,1,1,3))
    data = data.astype(np.float32)
    min = np.min(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) - min).transpose(3,0,1,2)
    max = np.max(data.reshape((data.shape[0], -1)), axis=1)
    data = (data.transpose(1,2,3,0) / max).transpose(3,0,1,2)
    # Inverting the colors seems to look better for MNIST
    if invert_colors:
       data = 1 - data

    n = int(np.ceil(np.sqrt(data.shape[0])))
    padding = ((0, n ** 2 - data.shape[0]), (0, 0),
            (0, 0)) + ((0, 0),) * (data.ndim - 3)
    data = np.pad(data, padding, mode='constant',
            constant_values=0)
    # Tile the individual thumbnails into an image.
    data = data.reshape((n, n) + data.shape[1:]).transpose((0, 2, 1, 3)
            + tuple(range(4, data.ndim + 1)))
    data = data.reshape((n * data.shape[1], n * data.shape[3]) + data.shape[4:])
    data = (data * 255).astype(np.uint8)
    return data, n

# Subsample the dataset
num_selected = 25*25
images_2d = np.array(images[:num_selected,:,:,:]).reshape(-1, 32, 32, 3)

# Create the sprite image
sprite, n = images_to_sprite(images_2d)
sprite_path = os.path.join(result_dir, f'{result_name}_sprite.png')
Image.fromarray(sprite).save(sprite_path)
print(f'Sprite image saved at: {sprite_path}')
#Image.fromarray(sprite).show()
fig = plt.figure(figsize=(10,10))
plt.imshow(sprite)
plt.axis('off')

## Projection embeddings

We load a pretrained network that will serve as a feature extractor. We choose mobilenetV2, which is a small and efficient network trained for classification on Imagenet. To use the network as a feature extractor, we remove the last layer (classification layer) and use the output of the previous layer. Other networks such as resnet and vgg can be used. 

### Load embedding model

In [ ]:
# Load the pre-trained image feature embedding model
embed = hub.load("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/feature_vector/5")

# required size for MobileNet v2
target_size = (224, 224)  
normalization = 255.0

### Create embeddings

To create the embeddings, first, we need to prepare the inputs as required by the model. For MobileNetv2, input images are required to be 224x224 pixels and normalized to the range [0,1]. Then, embeddings are created by projecting images through the model. 

In [ ]:
def normalize_images(images, normalization=255.0):
    images = images / normalization
    return images

def resize_images_to_tensors(images, image_size):
    # Convert to tf.float32 and expand the dimensions
    images_tensor = tf.cast(images, tf.float32)
    # Resize image
    images_tensor = tf.image.resize(images_tensor, size=image_size)
    return images_tensor

def create_embeddings_from_tensors(images_tensor, model):
    # Generate the image embedding
    embeddings = model(images_tensor)
    return embeddings

def create_embeddings_from_images(images, model, model_type='mobilenet'):
    if model_type == 'mobilenet':
        images = normalize_images(images, normalization=255.0)
        image_size = (224, 224)
    images_tensor = resize_images_to_tensors(images, image_size)
    embeddings = create_embeddings_from_tensors(images_tensor, model)
    return embeddings

# Create embeddings from a randomly selected number of images
idx = np.random.choice(len(images), num_selected, replace=False)
images_selected = images[idx]
labels_ids_selected = labels_ids[idx]
labels_selected = np.array(labels_names)[idx]

embeddings  = create_embeddings_from_images(images_selected, model=embed, model_type='mobilenet')
print(f'embeddings.shape: {embeddings.shape}, images.shape: {images_selected.shape}')

Embedded images have size $(N, 1280)$ where $N$ is the number of images and 1280 is the number of features in the last layer of the model. Thus, the embedding is a $1280$-dimensional space.

### Log to TensorBoard

Tensorboard provides experiment tracking, visualization and profiling. One can track metrics, model graph, projecting embeddings, and other images and results. For metrics, [tf.keras.callbacks](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard) are used. For an introduction to tensorboard, see or [PyTorch TensorBoard Tutorial](https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html). 

Here, we focus on the [TensorBoard Embedding Projector](https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin). Once the embeddings are created, we need to save a training checkpoint to a log directory `log_dir`, together with the medatada (labels) associated to each embedded data point. This is done in several steps:
1) Create a `tf.Variable` that holds the embedding (with name `'embedding'`). 
2) Create a checkpoint, `tf.train.Checkpoint`, with this variable and save it with the same name (`'embedding.ckpt'`). We also need to write the labels as metadata. 
3) Set up the projector `config` and add the embedding. 
4) Add `tensor_name` and `metadata_path` and write the labels to the metadata file. 
5) Add `sprite.image_path` to the embedding. 
6) Call `visualize_embeddings`. 

In [ ]:
# Assume embeddings is a 2D numpy array of shape (num_data_points, embedding_dim)
# and labels is a 1D numpy array of shape (num_data_points,) with the labels.

# Create a variable to hold the embeddings
embedding_var = tf.Variable(embeddings, name='embedding')

# Create and save a checkpoint for the embedding
checkpoint = tf.train.Checkpoint(embedding=embedding_var)
checkpoint.save(os.path.join(log_dir, 'embedding.ckpt'))

# Set up projector config
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`.
embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
embedding.metadata_path = 'metadata.tsv'

# Write labels to metadata file
metadata_path = os.path.join(log_dir, 'metadata.tsv')
with open(metadata_path, 'w') as metadata_file:
    for label in labels_selected:
        metadata_file.write(f'{label}\n')

# Add path to the sprite image
embedding.sprite.image_path = sprite_path
embedding.sprite.single_image_dim.extend([target_size[0], target_size[1]])
projector.visualize_embeddings(log_dir, config)
print(f'Projector config saved at: {os.path.abspath(log_dir)}')

In order to visualize the projection embedding, run tensorboard with the `log_dir` and access tensorboard on your browser via the provided URL. You may need to update the visualization on the top right. Run on the shell :
```
   tensorboard --logdir $log_dir
```

In [ ]:
# Run tensorboard 
if mode_colab:
    !tensorboard --logdir $log_dir

## Analysis of embeddings

Tensorboard provides an interactive visualization of the embedding. We can zoom in and out, select points, and see the corresponding images. We can also search for specific images and see the nearest neighbors. You can select 2D or 3D visualization, and images or labels. It provides three dimensionality reduction techniques: PCA, t-SNE, and UMAP.

### PCA embedding

The image below shows the PCA embedding of the CIFAR-10 dataset. It is a scatter plot of the first three principal components where each data point is an image taken from the sprite image. In this case, we have selected a red car (label 1). On the top right, it displays the 55 closest images using the cosine distance. Selecting only the 5 to 10 closest images, only red cars or red trucks are found. Increasing this number, other red objects or cars are found.

In [ ]:
# Display images for Medium blog
from IPython.display import Image
img = Image(filename=os.path.join(result_dir, 'cifar10_pca_car_selected.png'))
img.width = 800
img.height = 500
display(img)

On the top left, we can choose to display labels or images. The closest label to `1:'automobile'` is `9:'truck'`. Then, `0:'airplane'` and `8:'ship'`. The furthest labels are `7:'horse'` and `4:'deer`. Labels are `0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog', 6: 'frog', 7:'horse', 8: 'ship', 9:'truck'`.

In [ ]:
img = Image(filename=os.path.join(result_dir, 'cifar10_pca_car_selected_numbers.png'))
img.width = 800
img.height = 500
display(img)

The image below shows the UMAP embedding of the CIFAR-10 dataset. We can see that the classes are better separated than in the PCA embedding, as it is a non-linear embedding. Check the appendix section for a description of UMAP.

In [ ]:
img = Image(filename=os.path.join(result_dir, 'cifar10_umap_car_selected_numbers.png'))
img.width = 800
img.height = 500
display(img)

Below, we show the t-SNE embedding at different iteration numbers. You can play with the slider to tune the perplexity and learning rate. The perplexity is a measure of the effective number of neighbors. It is recommended to use a value between 5 and 50. The learning rate is the step size at each iteration. It is recommended to use a value between 10 and 1000. Then, you can stop the algorithm at the iteration number you want. As iterations increase, the classes are better separated. It is recommended to let it converge. However, after a certain number of iterations, images start to overlap within the same class. See below the t-SNE embedding at 1964, 3620, and 6336 iterations. At 3620 iterations, the classes are well separated and then start to overlap.

In [ ]:
# Load the images
filenames = [os.path.join(result_dir, f'cifar10_tsne_{i}_car_selected_numbers_red.png') for i in [1964, 3620, 6336]]

for filename in filenames:
    img = Image(filename=filename)
    img.width = 800
    img.height = 500
    display(img)



In TensorBoard, on the top bottom, there is a link to a recommendation on how to use t-SNE.

## Conclusions

In this notebook, we have explored a feature extractor and several dimensionality reduction techniques for data visualization in TensorBoard, but which one to use? One could generally start with PCA and then use t-SNE or UMAP if clusters are not well separated. However, we should stick to PCA if interpretability is important (singular vectors and singular values have a meaning while the dimensions of non-linear methods don't). In [McInnes 2020], authors argue that UMAP is a better choice than t-SNE, as it is faster and more robust.

We have chosen visualization in TensorBoard as provides out-of-the box techniques, it allows interacting with data points and is maintained and improved. Other posibility is to compute them using [SciKit-learn](https://scikit-learn.org/stable/auto_examples/manifold/plot_manifold_sphere.html#sphx-glr-auto-examples-manifold-plot-manifold-sphere-py) and then visualize them in TensorBoard or a Python library of your choice.

## Appendix: Theory

In this section, we summarize the theory behind the different embedding projection techniques for further understanding. 

Exploration of data structure via visualization in 2D or 3D is essential to uncover the underlying structure of the data. For this purpose, dimensionality reduction techniques are used. Dimensionality reduction techniques can be divided in two categories. In the first category, linear methods, such as PCA or non-negative matrix factorization (NMF), preserve pairwise distance among all data points (global distance). In the second category we have nonlinear methods, such as as Locally Linear Embedding (LLE), Isomap, Laplacian Eigenmaps, SNE, t-SNE, and UMAP, which favor to preserve local distance. 

Why do we need nonlinear methods? For high-dimensional data that lie on or near a lower-dimensional manifold, it is important that similar points are close together and dissimilar points are far apart, which is not possible with linear techniques [Van der Maaten 2008]. In the cases where the data is not linearly separable, nonlinear techniques are used. Non-linear dimensionality reduction techniques aim to separate the data into clusters or groups while maintaining the local distances between points (local manifold structure). While many of these methods exist, some of them cannot simultaneously preserve the local structure (distances between neighbors) and global structure (presence of clusters at several scales) of the data.

### PCA

PCA yields the best linear predictor of the data and it is commonly used for dimensionality reduction, whitening, and removing noise [Jolliffe 1986]. Here, we use it for dimensionality reduction and visualization purposes. PCA is a linear transformation that finds the directions of maximum variance in the data. The first principal component is the direction of maximum variance, the second principal component is the direction of maximum variance orthogonal to the first principal component, and so on. The principal components are the eigenvectors of the covariance matrix of the data. The eigenvalues of the covariance are the variance corresponding to the eigenvectors, so they can be seen as the spectrum of the covariance. 

Given the covariance matrix $C$, the eigenvectors $v_i$ and eigenvalues $\lambda_i$ are such that
$$
Cv_i=\lambda_i v_i.
$$
In matrix form, where $V=[v_1,\ldots,v_N]$ is an invertible orthogal matrix, 
$$
V^TCV=\textrm{diag}(\lambda_1,\ldots,\lambda_N).
$$

To understand the covariance and PCA, we can see how the covariance acts on vectors. If we see the covariance as a linear transformation, the eigenvectors provide the invariant directions of the transformation, and the eigenvalues the scalar factor by which they are transformed. Let's see the covariance in action! 

Given the points in a unit circle, the covariance will map them into an ellipse, where the axes correspond to the eigenvectors and their length to the eigenvalues:

In [ ]:
import math 

# Covariance
C = np.array([[3, 1], [1, 3]])
print(C)

# Points on the unit circle
sqrt2_inv = 1 / math.sqrt(2)
points = (0, 1), (1, 0), (0, -1), (-1, 0), (sqrt2_inv, sqrt2_inv), (-sqrt2_inv, sqrt2_inv), (-sqrt2_inv, -sqrt2_inv), (sqrt2_inv, -sqrt2_inv)

# Trnsform points by C
points_transformed = [np.dot(C, p) for p in points]

# Subplot the initial circle and the transformed one side by side
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

# Plot the initial circle
axs[0].scatter(*zip(*points))
axs[0].set_aspect('equal', 'box')
axs[0].set_title('Initial Circle')

# Plot the transformed circle
axs[1].scatter(*zip(*points_transformed))
axs[1].set_aspect('equal', 'box')
axs[1].set_title('Transformed Circle')

In this case, $v_1=(1/\sqrt(2), 1/\sqrt(2))$, $v_2=(-1/\sqrt(2), 1/\sqrt(2))$, $\lambda_1=4$ and $\lambda_2=2$.

For embedding CIFAR-10, the principal components represent the directions that explain the maximum variability across the data, being orthogonal to each other. Once computed, the principal components are found by projecting the data points onto the first three eigenvectors. 

### t-SNE

t-SNE (t-distributed Stochastic Neighbor Embedding) is a non-linear dimensionality reduction technique for visualization based on a previous technique called SNE, which tended to crowd points in the center of the plot. 

Stochastic Neighbor Embedding (SNE) first converts the high-dimensional Euclidean distances 
between datapoints into conditional probabilities that represent similarities. The similarity of datapoint $x_j$ to datapoint $x_i$ is the conditional probability, $p_{j|i}$, that $x_j$ would be picked as a neighbor of $x_i$ where the neighbors are picked by considering Gaussian distributions centered at $x_i$. The conditional probability is given by the following equation:
$$
p_{j|i} = \frac{\exp(-||x_i - x_j||^2 / 2\sigma_i^2)}{\sum_{k\neq i}\exp(-||x_i - x_k||^2 / 2\sigma_i^2)},
$$
where $\sigma_i$ is the variance of the Gaussian distribution centered on datapoint $x_i$, 
and $p_{i|i}=0$. For the low-dimensional map, the similarity of the corresponding points $y_i$ and $y_j$ is
$$
q_{j|i} = \frac{\exp(-||y_i - y_j||^2)}{\sum_{k\neq i}\exp(-||y_i - y_k||^2)},
$$
where the variance is set to $1/\sqrt{2}$ and $q_{i|i}=0$. 

SNE aims to find a low-dimensional representation of the data that minimizes the difference 
between $p_{j|i}$ and $q_{j|i}$, which is measured by the Kullback-Leibler divergence. 
Thus, the cost function is given by
$$
C=\sum_i KL(P_i||Q_i) = \sum_i \sum_j p_{j|i} \log\frac{p_{j|i}}{q_{j|i}},
$$
The parameter $\sigma_i$ must be chosen and is related to the perplexity, which is a measure 
of the effective number of neighbors. Typical values are between 5 and 50. 

SNE suffers from the crowding problem, where the points are crowded in the center of the plot, 
and by a cost function difficult to optimize. 
t-SNE uses a symmetrized version of the SNE and a Student-t distribution 
instead of a Gaussian distribution to compute the similarity between points. 
The heavy-tailed distribution diminishes these problems. 

In t-SNE, the joint probability $q_{ij}$ is given by
$$
q_{ij} = \frac{(1+||y_i - y_j||^2)^{-1}}{\sum_{k\neq l}(1+||y_k - y_l||^2)^{-1}},
$$
where a Student-t distribution with one degree of freedom is invariant to changes in scale. 
For more details see [Van der Maaten 2008]. 


### UMAP

UMAP (Uniform Manifold Approximation and Projection) belongs to class of k-neighbour based graph learning algorithms where we can include  Laplacian Eigenmaps, UMAP and t-SNE. In a first step, a weighted k-neighbor graph that represents the source space is constructed, and in a second step a low-dim approximation of this graph is computed while preserving the local distances. These methods favor preservation of local distances over long distances. 

The theory behind is quite complex. In brief, it first constructs a high-dimensional graph from the given data, where each node is connected to its k-nearest neighbors. Then, it constructs a low-dimensional graph that approximates the high-dimensional one using fuzzy simplicial sets. Its assumtions are that the data lies in a low-dimensional Riemannian manifold (smooth and curved space which allows to computed distances between two points along geodesics), the metric is locally constant and the manifold is uniformy distributed (points are spread evenly). The computational cost is bounded by k-NN calculation. Efficient optimization is done via stochastic gradient descent. 

See [McInnes 2020] for an interesting discussion on the effect of minimun distance, which expands clusters, and number of neighbors, which may depend on the expected number of clusters.

## References

* Tensorflow embedding projector tutorial: [Embedding Projector](https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin).

* I T Jolliffe. Principal component analysis. Springer-Verlag, New-York, 1986. 

* L McInnes, J Healy, J Melville. UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction, 2020

* L Van der Maaten, G Hinton. Visualizing Data using t-SNE. Journal of Machine Learning Research, 9, 2579-2605, 2008.